### Tarea 5 - pandas 
### Guillermo Jorge Naranjo Muedano 
### 190240

1. Carga de dato a pandas 

In [1]:
import pandas as pd
import numpy as np

In [2]:
casos_covid=pd.read_csv("/home/guillermo/Downloads/casos_nacionales_covid-19.csv")

In [3]:
casos_covid.drop(columns="Unnamed: 0",inplace=True)

In [4]:
casos_covid["fecha_ingreso"]=pd.to_datetime(casos_covid["fecha_ingreso"],yearfirst=True)
casos_covid["fecha_sintomas"]=pd.to_datetime(casos_covid["fecha_sintomas"],yearfirst=True)
casos_covid["fecha_def"]=pd.to_datetime(casos_covid["fecha_def"],yearfirst=True)

2. Genera un dataframe defunciones en que solo existan registro que tengan una fecha de defunción.

In [5]:
defunciones=casos_covid[casos_covid.fecha_def.isnull() == False]

* Cuántos registros hay por entidad de residencia `entidad_res`, regresa una sola columna que se llama `num_defunciones`

In [6]:
#Solo suman 16765, en los demás casos la entidad_res es na
defunciones.groupby(["entidad_res"]).size().reset_index(name="num_defunciones")

,entidad_res,num_defunciones
0,GUANAJUATO,53
1,GUERRERO,162
2,HIDALGO,472
3,JALISCO,11
4,MICHOACÁN DE OCAMPO,71
5,MORELOS,149
6,MÉXICO,15217
7,NAYARIT,1
8,NUEVO LEÓN,10
9,OAXACA,75


* Cuál es el top 10 de entidades con mayor número de defunciones?

In [7]:
defunciones.groupby(["entidad_res"]).size().reset_index(name="num_defunciones")\
.sort_values(by="num_defunciones",ascending=False).head(10)

,entidad_res,num_defunciones
6,MÉXICO,15217
2,HIDALGO,472
10,PUEBLA,258
1,GUERRERO,162
5,MORELOS,149
19,VERACRUZ DE IGNACIO DE LA LLAVE,98
9,OAXACA,75
4,MICHOACÁN DE OCAMPO,71
18,TLAXCALA,67
0,GUANAJUATO,53


3. Acomoda los datos para que las columnas: neumonia, embarazo, diabetes, epoc, asma, inmusupr, hipertension, otra_com, cardiovascular, obesidad, renal_cronica, tabaquismo, otro_caso en una sola columna comorbilidad y la columna de su valor sea valor.

In [8]:
columnas_melt=["neumonia","embarazo","diabetes","epoc","asma","inmusupr","hipertension",\
          "otra_com","cardiovascular","obesidad","renal_cronica","tabaquismo","otro_caso"]
id_melt=['fecha_actualizacion', 'id_registro', 'origen', 'sector',
       'entidad_um', 'sexo', 'entidad_nac', 'entidad_res',
       'municipio_res', 'tipo_paciente', 'fecha_ingreso',
       'fecha_sintomas', 'fecha_def','edad','nacionalidad','indigena',\
         'toma_muestra_lab', 'resultado_lab',
       'toma_muestra_antigeno', 'resultado_antigeno',
       'clasificacion_final', 'migrante', 'pais_nacionalidad',
       'pais_origen', 'uci']

In [9]:
defunciones_long=defunciones.melt(id_vars=id_melt,value_vars=columnas_melt,\
                                  var_name="comorbilidad")

* Cuántos registros de defunciones tienes por `comorbilidad`?

In [10]:
defunciones_long[defunciones_long.value=="SI"].groupby(["comorbilidad"]).size().reset_index(name="casos_por_comorbilidad")

,comorbilidad,casos_por_comorbilidad
0,asma,816
1,cardiovascular,3308
2,diabetes,21562
3,embarazo,38
4,epoc,2795
5,hipertension,25173
6,inmusupr,1853
7,neumonia,47188
8,obesidad,12707
9,otra_com,3443


* Cuántos registros de defunciones tienes por `entidad` y `comorbilidad`?

In [11]:
defunciones_long[defunciones_long.value=="SI"].groupby(["comorbilidad","entidad_res"]).size().reset_index(name="casos_por_comorbilidad")

,comorbilidad,entidad_res,casos_por_comorbilidad
0,asma,GUANAJUATO,1
1,asma,GUERRERO,2
2,asma,HIDALGO,4
3,asma,JALISCO,1
4,asma,MICHOACÁN DE OCAMPO,2
...,...,...,...
198,tabaquismo,SINALOA,1
199,tabaquismo,TAMAULIPAS,3
200,tabaquismo,TLAXCALA,8
201,tabaquismo,VERACRUZ DE IGNACIO DE LA LLAVE,6


4. casos de sobrevivientes

In [12]:
#Solo tomamos a los que no tienen fecha de defunción y dieron positivo a COVID-19
sobrevivientes=casos_covid[(casos_covid.fecha_def.isnull() == True) & (casos_covid.resultado_antigeno == 'POSITIVO A SARS-COV-2')]

In [13]:
sobrevivientes_long=sobrevivientes.melt(id_vars=id_melt,value_vars=columnas_melt,\
                                  var_name="comorbilidad")

In [14]:
sobrevivientes['intubado'].value_counts(normalize=True)

NO APLICA    0.979025
NO           0.020481
SI           0.000494
Name: intubado, dtype: float64

* Cuántos registros hay por `entidad` y `comorbilidad`.

In [15]:
sobrevivientes_long[sobrevivientes_long.value == 'SI'].groupby(['entidad_res','comorbilidad']).size().reset_index(name='casos_entidad_comorbilidad')

,entidad_res,comorbilidad,casos_entidad_comorbilidad
0,DURANGO,hipertension,3
1,DURANGO,otro_caso,4
2,GUANAJUATO,asma,1
3,GUANAJUATO,cardiovascular,4
4,GUANAJUATO,diabetes,12
...,...,...,...
200,ZACATECAS,diabetes,1
201,ZACATECAS,hipertension,2
202,ZACATECAS,obesidad,1
203,ZACATECAS,otro_caso,6


5. Genera una columna `días_hospitalizado` que obtenga para las defunciones los tipos de paciente `HOSPITALIZADO` las siguientes estadísticas interpretando el significado de cada una:

In [16]:
#Usamos el df de defunciones para crear uno nuevo con la columna dias_hospitalizado
defunciones_con_dia=defunciones[defunciones.tipo_paciente == 'HOSPITALIZADO'].assign(dias_hospitalizado=defunciones.fecha_def-defunciones.fecha_ingreso)

In [17]:
defunciones_con_dia['dias_hospitalizado'].describe()

count                      60784
mean      9 days 00:30:03.790471
std      11 days 14:52:08.022549
min           -31 days +00:00:00
25%              3 days 00:00:00
50%              6 days 00:00:00
75%             12 days 00:00:00
max            430 days 00:00:00
Name: dias_hospitalizado, dtype: object

* Como el minímo de la muestra es negativo, asumo que en algunos casos la captura de datos fue errónea, usaré solamente los registros con `dias_hospitalizado`> '0 days'

In [18]:
defunciones_con_dia[defunciones_con_dia.dias_hospitalizado > '0 days 00:00:00']['dias_hospitalizado'].describe()

count                      55762
mean      9 days 20:08:55.590545
std      11 days 18:55:04.098218
min              1 days 00:00:00
25%              3 days 00:00:00
50%              7 days 00:00:00
75%             13 days 00:00:00
max            430 days 00:00:00
Name: dias_hospitalizado, dtype: object

* count: tenemos 55762 casos en la muestra, eliminando los que son menores a cero días 
* mean: la media (promedio) de días que los pacientes permanecieron hospitalizados antes de fallecer es de 9 días 20 horas 
* std: nos dice que la mayor parte de la probabilidad esta acumulada en 9 días +- 11 días (muy grande por valores extremos)
* min: el paciente que menos tiempo estuvo hospitalizado fue 1 día 
* 25%: 3 días es el valor donde la probabilidad acumulada llega a 25%
* 50%: 7 días es el valor donde la probabilidad acumulada llega a 50%
* 75%: 13 días es el valor donde la probabilidad acumulada llega a 75%
* max: 430 días fue el número máximo de días que un paciente estuvo intubado (¿error?)